<a href="https://colab.research.google.com/github/Kerriea-star/Distributed-training/blob/main/Distributed_training_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
The `tf.distribute.Strategy` API provides an abstraction for distributing your training across multiple processing units. It allows you to carry out distributed training using existing models and training code with minimal changes.

This project demonstrates how to use the `tf.distribute.MirroredStrategy` to perform in-graph replication with synchronous training on many GPUs on one machine. The strategy essentially copies all of the model's variables to each processor. Then, it uses **all-reduce** to combine the gradients from all processors, and applies the combined value to all copies of the model.

You will use the `tf.keras` APIs to build the model and `Model.fit` for training it.

`MirroredStrategy` trains your model on multiple GPUs on a single machine. For synchronous training on many GPUs on multiple workers, use the `tf.distribute.MultiWorkerMirroredStrategy` with the Keras Model.fit or a custom training loop.

## Setup

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
print(tf.__version__)

2.13.0


## Download the dataset
Load the MNIST dataset from **TensorFlow Datasets**. This returns a dataset in the `tf.data` format.

Setting the `with_info` argument to `True` includes the metadata for the entire dataset, which is being saved here to `info`. Among other things, this metadata object includes the number of train and test examples.

In [3]:
datasets, info  = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets["train"], datasets["test"]

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


## Define the distribution strategy
Create a `MirroredStrategy` object. This will handle distribution and provide a context manager (`MirroredStrategy.scope`) to build your model inside.

In [4]:
strategy = tf.distribute.MirroredStrategy()

In [6]:
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

Number of devices: 1
